# Ejecución de los Modelos de Regresión

## Librerías que utilizaremos

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split, KFold, RepeatedKFold, cross_val_predict
from sklearn.preprocessing import StandardScaler, KBinsDiscretizer
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor

In [3]:
datos = pd.read_csv('data/data_regresion.csv')

### Diseño de la Clase `Metrics` para la Evaluación Centralizada (Regresión)

Para gestionar y reportar de forma consistente los resultados de múltiples ejecuciones (distintas configuraciones de validación y modelos de **regresión**), se utiliza la clase `Metrics`.

### Estructura y funcionamiento

* **Diccionario central (`results = {}`)**

  * Se define un diccionario a nivel de clase (`Metrics.results`) como registro global centralizado.
  * Cada ejecución guarda sus resultados bajo una clave única `validation_name` (p. ej., `"ZeroR Holdout simple"`, `"KNN(k=5) CV-10fold"`, `"MyKNNRegressor (k=3) RepeatedKFold 5x2"`).
  * Esto permite la **agregación posterior** (p. ej., medias en 10 holdouts o evaluación global tras concatenar predicciones).

* **Métricas individuales (métodos)**

  * La clase implementa, cada una en su método, las métricas estándar de regresión:

    * `mse()` – Error cuadrático medio.
      $ \mathrm{MSE}=\frac{1}{n}\sum_{i=1}^{n}\big(y_i-\hat{y}_i\big)^2$

    * `mae()` – Error absoluto medio.
      $ \mathrm{MAE}=\frac{1}{n}\sum_{i=1}^{n}\big|y_i-\hat{y}_i\big|$

    * `rmse()` – Raíz del MSE.
      $ \mathrm{RMSE}=\sqrt{\frac{1}{n}\sum_{i=1}^{n}\big(y_i-\hat{y}_i\big)^2}$

    * `r2()` – Coeficiente de determinación.
      $ R^2=1-\frac{\sum_{i=1}^{n}\big(y_i-\hat{y}*i\big)^2}{\sum*{i=1}^{n}\big(y_i-\bar{y}\big)^2}$

    * `mape()` – Error porcentual absoluto medio.
      $ \mathrm{MAPE}(\%)=\frac{100}{n}\sum_{i=1}^{n}\left|\frac{y_i-\hat{y}_i}{,|y_i|+\varepsilon,}\right|)$

    * `rmspe()` – Raíz del error porcentual cuadrático medio.
      $ \mathrm{RMSPE}(\%)=100\sqrt{\frac{1}{n}\sum_{i=1}^{n}\left(\frac{y_i-\hat{y}_i}{,|y_i|+\varepsilon,}\right)^2}$
  * Para evitar divisiones por cero en métricas porcentuales, la clase usa un **pequeño `eps`** (p. ej., `1e-12`) en los denominadores.

* **Normalización de entradas**

  * En el constructor, `y_true` y `y_pred` se convierten a `np.array` de tipo `float` y se aplastan a 1D, evitando problemas de tipos (Series/DataFrames) y garantizando cálculos consistentes.

* **Método principal `all_metrics(self)`**

  1. **Cálculo**: invoca todos los métodos y compone un diccionario `calculated_metrics` con `MSE`, `MAE`, `RMSE`, `R2`, `MAPE`, `RMSPE`.
  2. **Almacenamiento**: guarda `calculated_metrics` en `Metrics.results[validation_name]`.
  3. **Impresión**: muestra un resumen legible en consola (valores en notación decimal; `MAPE` y `RMSPE` en **%**).


In [5]:

class Metrics:
    # Diccionario a nivel de clase para almacenar todos los resultados de todas las ejecuciones
    results = {}

    def __init__(self, y_true, y_pred, validation_name, eps=1e-12):
        # Aseguramos arrays 1D float para evitar problemas de tipos/Series
        self.y_true = np.asarray(y_true, dtype=float).ravel()
        self.y_pred = np.asarray(y_pred, dtype=float).ravel()
        self.validation_name = validation_name
        self.eps = float(eps)

    # --- Métricas de regresión ---
    def mse(self):
        return mean_squared_error(self.y_true, self.y_pred)

    def mae(self):
        return mean_absolute_error(self.y_true, self.y_pred)

    def rmse(self):
        return np.sqrt(self.mse())

    def r2(self):
        return r2_score(self.y_true, self.y_pred)

    def mape(self):
        # *100 para porcentaje; eps evita divisiones por cero
        return np.mean(np.abs((self.y_true - self.y_pred) / (np.abs(self.y_true) + self.eps))) * 100.0

    def rmspe(self):
        return np.sqrt(np.mean(((self.y_true - self.y_pred) / (np.abs(self.y_true) + self.eps)) ** 2)) * 100.0

    # Para obtener todas las métricas de una vez, imprimirlas y almacenarlas
    def all_metrics(self):
        calculated_metrics = {
            'MSE': self.mse(),
            'MAE': self.mae(),
            'RMSE': self.rmse(),
            'R2': self.r2(),
            'MAPE': self.mape(),
            'RMSPE': self.rmspe()
        }

        # Guardar resultados
        Metrics.results[self.validation_name] = calculated_metrics

        # Imprimir con formato
        print(f"\n===== Resultados {self.validation_name} =====")
        for k, v in calculated_metrics.items():
            if k in ('MAPE', 'RMSPE'):
                print(f'{k}: {v:.4f}%')
            else:
                print(f'{k}: {v:.6f}')


## Holdout simple 75-25

In [4]:
# Dividimos los datos en conjunto de entrenamiento y prueba
X = datos.drop('Popularity', axis=1)
y = datos['Popularity']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)


### Linear Regresion

In [6]:
lr = LinearRegression()
lr.fit(X_train, y_train)
y_pred_lr = lr.predict(X_test)
metrics_lr = Metrics(y_test, y_pred_lr, 'LinearRegression Holdout simple')
metrics_lr.all_metrics()


===== Resultados LinearRegression Holdout simple =====
MSE: 283.877226
MAE: 13.426346
RMSE: 16.848656
R2: 0.065825
MAPE: 71.8222%
RMSPE: 310.8880%


### Decision Tree Regressor

In [9]:
lr = DecisionTreeRegressor(min_samples_leaf=1, ccp_alpha=0.0, random_state=42)
lr.fit(X_train, y_train)
y_pred_lr = lr.predict(X_test)
metrics_lr = Metrics(y_test, y_pred_lr, 'DecisionTreeRegressor Holdout simple')
metrics_lr.all_metrics()


===== Resultados DecisionTreeRegressor Holdout simple =====
MSE: 461.807377
MAE: 15.711976
RMSE: 21.489704
R2: -0.519702
MAPE: 67.9508%
RMSPE: 290.2679%


## 10 Holdout repetidos 75-25

In [10]:
# Dividimos los datos en 10 conjuntos de entrenamiento y pruebas

# 1. Inicializar listas para almacenar los 10 conjuntos
X_train_list = []
X_test_list = []
y_train_list = []
y_test_list = []

# 2. Bucle para crear 10 divisiones y escalados diferentes
NUM_SPLITS = 10

for i in range(NUM_SPLITS):
    # a. División: Usamos 'i' como random_state para asegurar 10 divisiones distintas
    X_train_temp, X_test_temp, y_train_temp, y_test_temp = train_test_split(
        X, y, 
        test_size=0.25, 
        random_state=i # CLAVE: Usar 'i' para tener 10 divisiones diferentes
    )
    
    # b. Estandarización
    scaler = StandardScaler()
    
    #  fit_transform en Train
    X_train_scaled = scaler.fit_transform(X_train_temp)
    
    # transform en Test (usando los parámetros aprendidos en Train)
    X_test_scaled = scaler.transform(X_test_temp)
    
    # c. Almacenar los resultados en las listas
    X_train_list.append(X_train_scaled)
    X_test_list.append(X_test_scaled)
    y_train_list.append(y_train_temp)
    y_test_list.append(y_test_temp)

 

### Linear Regresion

In [12]:
for i in range(NUM_SPLITS):
    # a. Obtener los conjuntos de entrenamiento y prueba actuales
    X_train_current = X_train_list[i]
    X_test_current = X_test_list[i]
    y_train_current = y_train_list[i]
    y_test_current = y_test_list[i]
    
    # b. Entrenar el regresor LinearRegression
    lr = LinearRegression()
    lr.fit(X_train_current, y_train_current)
    y_pred_lr = lr.predict(X_test_current)
    

    # d. Metricas para el holdout actual
    metrics_10holdout_lr = Metrics(y_test_current, y_pred_lr, f'LinearRegression Holdout {i+1}')
    metrics_10holdout_lr.all_metrics()


# Calcular la precisión media y desviación estándar


# Calcular las métricas promedio para los 10 holdouts de LinearRegression

# Del diccionario results extraer las métricas y calcular la media para cada una
# MSE
all_mse = [Metrics.results[f'LinearRegression Holdout {i+1}']['MSE'] for i in range(NUM_SPLITS)]
mse_mean_10holdout = np.mean(all_mse)
# MAE
all_mae = [Metrics.results[f'LinearRegression Holdout {i+1}']['MAE'] for i in range(NUM_SPLITS)]
mae_mean_10holdout = np.mean(all_mae)
# RMSE
all_rmse = [Metrics.results[f'LinearRegression Holdout {i+1}']['RMSE'] for i in range(NUM_SPLITS)]
rmse_mean_10holdout = np.mean(all_rmse)
# R2
all_r2 = [Metrics.results[f'LinearRegression Holdout {i+1}']['R2'] for i in range(NUM_SPLITS)]
r2_mean_10holdout = np.mean(all_r2)
# MAPE
all_mape = [Metrics.results[f'LinearRegression Holdout {i+1}']['MAPE'] for i in range(NUM_SPLITS)]
mape_mean_10holdout = np.mean(all_mape)
# RMSPE
all_rmspe = [Metrics.results[f'LinearRegression Holdout {i+1}']['RMSPE'] for i in range(NUM_SPLITS)]
rmspe_mean_10holdout = np.mean(all_rmspe)


# Imprimir las métricas promedio
print('\n===== Métricas promedio LinearRegression en 10 Holdouts =====')
print(f'MSE medio: {mse_mean_10holdout:.4f}')
print(f'MAE medio: {mae_mean_10holdout:.4f}')
print(f'RMSE medio: {rmse_mean_10holdout:.4f}')
print(f'R2 medio: {r2_mean_10holdout:.4f}')
print(f'MAPE medio: {mape_mean_10holdout:.4f}')
print(f'RMSPE medio: {rmspe_mean_10holdout:.4f}')


# 1. Crear el diccionario de resultados medios
mean_results = {
    'MSE':   mse_mean_10holdout,
    'MAE':   mae_mean_10holdout,
    'RMSE':  rmse_mean_10holdout,
    'R2':    r2_mean_10holdout,
    'MAPE':  mape_mean_10holdout,
    'RMSPE': rmspe_mean_10holdout
}

# 2. Guardar en Metrics.results con una clave única
Metrics.results['LinearRegression_10_Holdouts_MEAN'] = mean_results





===== Resultados LinearRegression Holdout 1 =====
MSE: 288.519944
MAE: 13.625251
RMSE: 16.985875
R2: 0.059033
MAPE: 74.8759%
RMSPE: 336.3728%

===== Resultados LinearRegression Holdout 2 =====
MSE: 281.851387
MAE: 13.382224
RMSE: 16.788430
R2: 0.064059
MAPE: 67.7704%
RMSPE: 316.7796%

===== Resultados LinearRegression Holdout 3 =====
MSE: 288.170784
MAE: 13.625138
RMSE: 16.975594
R2: 0.057237
MAPE: 66.6737%
RMSPE: 295.2983%

===== Resultados LinearRegression Holdout 4 =====
MSE: 285.001991
MAE: 13.481445
RMSE: 16.882002
R2: 0.049867
MAPE: 65.8967%
RMSPE: 280.1469%

===== Resultados LinearRegression Holdout 5 =====
MSE: 288.119232
MAE: 13.612096
RMSE: 16.974075
R2: 0.053589
MAPE: 71.0001%
RMSPE: 309.9992%

===== Resultados LinearRegression Holdout 6 =====
MSE: 288.024352
MAE: 13.535077
RMSE: 16.971280
R2: 0.058895
MAPE: 74.6297%
RMSPE: 323.6979%

===== Resultados LinearRegression Holdout 7 =====
MSE: 286.004660
MAE: 13.562987
RMSE: 16.911672
R2: 0.053908
MAPE: 65.8629%
RMSPE: 277.1906%